In [ ]:
import matplotlib as mpl
import numpy as np
import pandas as pd
from pathlib import Path

mpl.use("pdf")
import matplotlib.pyplot as plt
from gsd_utils import papermode

papermode(plt=plt, size=8, has_latex=True)

# width as measured in inkscape
width = 3.281
height = width / 1.618

# Path
figdir = Path("figs/phase2")
resdir = Path("results/New_phase_2")
resvdir = resdir / "stride"
respdir = resdir / "voting-patch"


In [ ]:
# Different step sizes
dfv_val_metr = pd.read_csv(resvdir / "val/phase2-val-stride_METRICS.csv")
dfv_val_vote = pd.read_csv(resvdir / "val/phase2-val-stride.csv")
dfv_test_metr = pd.read_csv(resvdir / "test/phase2-test-stride_METRICS.csv")
dfv_test_vote = pd.read_csv(resvdir / "test/phase2-test-stride.csv")

dfv_val = pd.merge(
    dfv_val_metr,
    dfv_val_vote,
    how="outer",
    on=["Central Size", "Patch Size", "Step Size", "Pad Size"],
)
dfv_test = pd.merge(
    dfv_test_metr,
    dfv_test_vote,
    how="outer",
    on=["Central Size", "Patch Size", "Step Size", "Pad Size"],
)
dfv_filt_val = dfv_val[(dfv_val["Pad Size"] == 184)]
dfv_filt_test = dfv_test[(dfv_test["Pad Size"] == 184)]
dfv_filt_val.loc[:, "F1"] = 100 * dfv_filt_val.F1
dfv_filt_test.loc[:, "F1"] = 100 * dfv_filt_test.F1

# Entire vs Center
dfp_val_metr = pd.read_csv(respdir / "val/phase2-val-metrics.csv")
dfp_val_vote = pd.read_csv(respdir / "val/phase2-val-votes.csv")
dfp_test_metr = pd.read_csv(respdir / "test/phase2-test-metrics.csv")
dfp_test_vote = pd.read_csv(respdir / "test/phase2-test-votes.csv")

dfp_val = pd.merge(
    dfp_val_metr,
    dfp_val_vote,
    how="outer",
    on=["Central Size", "Patch Size", "Step Size", "Pad Size"],
)
dfp_test = pd.merge(
    dfp_test_metr,
    dfp_test_vote,
    how="outer",
    on=["Central Size", "Patch Size", "Step Size", "Pad Size"],
)
dfp_filt_val = dfp_val[(dfp_val["Pad Size"] == 184) & (dfp_val["Step Size"] == 27)]
dfp_filt_test = dfp_test[(dfp_test["Pad Size"] == 184) & (dfp_test["Step Size"] == 27)]
dfp_filt_val.loc[:, "F1"] = 100 * dfp_filt_val.F1
dfp_filt_test.loc[:, "F1"] = 100 * dfp_filt_test.F1


In [ ]:
# fig = plt.figure(layout="constrained")
# subfigs = fig.subfigures(ncols=2)
# ax1 = subfigs[0].subplots()
# ax2 = subfigs[1].subplots()
layout = [
    ["A", "."],
    ["A", "."],
    ["A", "B"],
    ["A", "B"],
    ["A", "B"],
    ["A", "B"],
    ["A", "B"],
]

fig = plt.figure(layout="constrained")
axs = fig.subplot_mosaic(layout)

ax1 = axs["A"]
ax2 = axs["B"]

# fig, (ax1, ax2) = plt.subplots(ncols=2)

# ===============================

# Different step sizes
ax1.plot(
    dfv_filt_val.Avg_Voters,
    dfv_filt_val.F1,
    marker="o",
    linestyle="-",
    color="xkcd:irish green",
    label=r"$D_{val}^{drone}$",
)
ax1.plot(
    dfv_filt_test.Avg_Voters,
    dfv_filt_test.F1,
    marker="o",
    linestyle="-",
    color="xkcd:lime",
    label=r"$D_{test}^{drone}$",
)

ax1.set_xlabel("Average number of voters")
ax1.xaxis.set_label_coords(0.5, -0.19)

ax1.set_ylabel("$F1$ score (\\%)")
ax1.yaxis.set_label_coords(-0.17, 0.5)

ax1.yaxis.set_major_locator(plt.MultipleLocator(2))
ax1.yaxis.set_minor_locator(plt.MultipleLocator(1))

ax1.grid(True, which="major", axis="both")
ax1.grid(True, which="minor", axis="y", linestyle="--")

# ===============================

# Entire vs step size
bar_width = 0.45
labels = dfp_filt_val["Central Size"].astype(str)
x1 = np.arange(len(labels))
x2 = x1 + bar_width

bars1 = ax2.bar(
    x1,
    dfp_filt_val.F1,
    color="xkcd:irish green",
    edgecolor="black",
    width=bar_width,
    label=r"$D_{val}^{drone}$",
)
bars2 = ax2.bar(
    x2,
    dfp_filt_test.F1,
    color="xkcd:lime",
    edgecolor="black",
    width=bar_width,
    label=r"$D_{test}^{drone}$",
)

ax2.bar_label(bars1, padding=2, fontsize="x-small", color="darkgreen", zorder=3)
ax2.bar_label(bars2, padding=2, fontsize="x-small", color="darkgreen", zorder=3)

ax2.set_xlabel("Central Window Size")
ax2.xaxis.set_label_coords(0.5, -0.25)

ax2.set_ylabel("$F1$ score (\%)")
ax2.yaxis.set_label_coords(-0.25, 0.5)

ax2.set_xticks(x1 + (bar_width / 2), labels)
ax2.yaxis.set_major_locator(plt.MultipleLocator(0.5))

ylim = (
    min(dfp_filt_val.F1.min(), dfp_filt_test.F1.min()) - 0.5,
    max(dfp_filt_val.F1.max(), dfp_filt_test.F1.max()) + 0.5,
)
ax2.set_ylim(ylim)

h, l = ax2.get_legend_handles_labels()
fig.legend(
    handles=h,
    labels=l,
    ncols=2,
    # loc="lower center",
    # bbox_to_anchor=(0.5, 0.86),
    labelcolor="b",
)

# ===============================

fig.subplots_adjust(top=0.95, bottom=0.21, left=0.1, right=0.99, wspace=0.4)
fig.set_size_inches(width, 0.8 * height)

fig.savefig("figs/phase2/phase2-voting.png")
fig.savefig("figs/phase2/phase2-voting.pdf")